<a href="https://colab.research.google.com/github/1337DS/Party-Barometer/blob/teil2_einzelwertung/Webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
from datetime import datetime, timedelta


# Abfrage Regen

In [4]:
def calc_regenscore():
  """Frägt die Wettervorhersage für Mannheim im 3-Std Rhythmus ab und gibt zurück, an wievielen Zeitpunkten zwischen heute 18:00 
  und morgen 03:00 Uhr die Wettervorhersage Regen beinhaltet"""
  api_key = "ca43b31915cb2b6af64970e1835b94d6"  # Bitte nicht missbrauchen
  base_url = "http://api.openweathermap.org/data/2.5/forecast?"

  #city_name = input("Enter city name : ")
  city_name="Mannheim"
  complete_url = base_url + "appid=" + 'd850f7f52bf19300a9eb4b0aa6b80f0d' + "&q=" + city_name  # Vervollständigung der URL
  response = requests.get(complete_url)
  antwort_json = response.json()


  if antwort_json["cod"] != "404":
      regenscore=0
      for vorhersage_punkt in antwort_json["list"]:
        #print(vorhersage_punkt["dt_txt"][-12:])
        kalendertag = datetime.today().strftime("%d")
        kalendertag_morgen = datetime.today() + timedelta(days=1)
        kalendertag_morgen=kalendertag_morgen.strftime("%d")
        
        if (vorhersage_punkt["dt_txt"][-11:] in [kalendertag+' 18:00:00',kalendertag+' 21:00:00'])or vorhersage_punkt["dt_txt"][-11:] in [kalendertag_morgen+' 00:00:00',kalendertag_morgen+' 03:00:00']:
        
        #if vorhersage_punkt["dt_txt"][-11:] in [kalendertag_morgen+ ' 00:00:00',kalendertag_morgen+' 03:00:00']:

        # evtl kann man hier noch die Temperatur mit einbeziehen
        # z.B. wenn es viel zu heiss oder zu kalt ist
        # current_temperature = y["temp"]
        
          wetterdaten = vorhersage_punkt["weather"]
          weather_description = wetterdaten[0]["description"]
      #    weather_main = wetterdaten[0]["main"]

          print(vorhersage_punkt["dt_txt"]+" Wetter (Beschreibung) = " +
                          str(weather_description))
                          # " Wetter (main) = " +
                          # str(weather_main))
    
          if 'rain' in weather_description:
              regenscore+=1
              print(regenscore)

        return regenscore 

  else:
      print(" Fehler bei Abfrage der Wetterdaten")

In [5]:
b=calc_regenscore()

In [6]:
print(b)

0


# Abfrage Events

In [27]:
from bs4 import BeautifulSoup
import requests

#Webscraping Club Chalpin
URL = 'https://www.chaplin-club.de/event/' 
chaplin = requests.get(URL)
soup_chaplin = BeautifulSoup(chaplin.text, 'html.parser')
list=soup_chaplin.find_all('time',{'class': 'tribe-events-calendar-list__event-date-tag-datetime'})
list_date=soup_chaplin.find_all('span',{'class': 'tribe-events-calendar-list__event-date-tag-daynum'})
list_days=[]

#Filtern der Daten
for span in list_date:
  list_days.append(span.text)
list_numbers =[]
for i in list_days:
  list_numbers.append([int(temp)for temp in i.split() if temp.isdigit()])
dates=[]
for i in range(len(list_numbers)):
  dates.append(list_numbers[i][0])

from datetime import datetime,date

#Anpassung der Daten mit Monat und Jahr
month = datetime.now().month
year = datetime.now().year


event_dates=[]
checker = True
for i in range(len(dates)-1):
  if dates[i]<dates[i+1]:
    if checker == True:
      dummy = str(dates[i])+'-'+str(month)+'-'+str(year)
      dummy_2 = datetime.strptime(dummy,'%d-%m-%Y').date()
      event_dates.append(dummy_2)
  else:
      checker = False

#Überprüfung der Eventzahl --> Kann noch durch weitere Clubs erweitert werden 
today = date.today()
events = 0
for i in range(len(event_dates)):
  if event_dates[i] == today:
    events +=1

In [8]:
events

1

# Abfrage Studenten

In [9]:
!wget https://www.uni-mannheim.de/media/Universitaet/Dokumente/Studierendenstatistik/2020-/Studierendenstatistik_FSS_2022_UniMA.pdf -O StudentenUni.pdf 
!pip install pdfplumber

--2022-08-12 02:39:54--  https://www.uni-mannheim.de/media/Universitaet/Dokumente/Studierendenstatistik/2020-/Studierendenstatistik_FSS_2022_UniMA.pdf
Resolving www.uni-mannheim.de (www.uni-mannheim.de)... 134.155.101.1, 2001:7c0:2900:64::869b:6501
Connecting to www.uni-mannheim.de (www.uni-mannheim.de)|134.155.101.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 968335 (946K) [application/pdf]
Saving to: ‘StudentenUni.pdf’

StudentenUni.pdf    100%[===================>] 945.64K  1.02MB/s    in 0.9s    

2022-08-12 02:39:56 (1.02 MB/s) - ‘StudentenUni.pdf’ saved [968335/968335]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 4.4 MB/s 
     |████████████████████████████████| 3.1 MB 16.1 MB/s 
     |████████████████████████████████| 5.6 MB 36.4 MB/s 
     |████████████████████████████████| 142 kB 53.4 MB/s 
     |████████████████████████████████| 4.1 MB 41.4 MB/s 


In [10]:


import pdfplumber
from os import replace
with pdfplumber.open('StudentenUni.pdf') as pdf:
    gesamtstatistik = pdf.pages[4]
    print(gesamtstatistik.extract_tables())

dict_anzahl_studenten={}
dict_anzahl_studenten['jura_ges']=int(gesamtstatistik.crop((408.53, 194.33000000000004, 433.67911999999995, 205.37)).extract_text().replace(".",""))
dict_anzahl_studenten['jura_w']=int(gesamtstatistik.crop((473.59, 194.33000000000004, 498.73912, 205.37)).extract_text().replace(".",""))
dict_anzahl_studenten['vwl_ges']=int(gesamtstatistik.crop((408.53, 210.77, 433.67911999999995, 221.81)).extract_text().replace(".",""))
dict_anzahl_studenten['vwl_w']=int(gesamtstatistik.crop((473.59, 210.77, 498.73912, 221.81)).extract_text().replace(".",""))
dict_anzahl_studenten['bwl_ges']=int(gesamtstatistik.crop((408.53, 227.81000000000006, 434, 238.85000000000002)).extract_text().replace(".",""))
dict_anzahl_studenten['bwl_w']=int(gesamtstatistik.crop((473.59, 227.81000000000006, 498.73912, 238.85000000000002)).extract_text().replace(".",""))
dict_anzahl_studenten['sozi_ges']=int(gesamtstatistik.crop((408.53, 244.73000000000008, 434, 255.77000000000004)).extract_text().replace(".",""))
dict_anzahl_studenten['sozi_w']=int(gesamtstatistik.crop((473.59, 244.73000000000008, 498.73912, 255.77000000000004)).extract_text().replace(".",""))
dict_anzahl_studenten['philo_ges']=int(gesamtstatistik.crop((408.53, 261.77000000000004, 434, 272.81)).extract_text().replace(".",""))
dict_anzahl_studenten['philo_w']=int(gesamtstatistik.crop((473.59, 261.77000000000004, 498.73912, 272.81)).extract_text().replace(".",""))
dict_anzahl_studenten['wiinf_ges']=int(gesamtstatistik.crop((408.53, 278.81000000000006, 434, 289.85)).extract_text().replace(".",""))
dict_anzahl_studenten['wiinf_w']=int(gesamtstatistik.crop((473.59, 278.81000000000006, 498.73912, 289.85)).extract_text().replace(".",""))  

[]


In [11]:
dict_anzahl_studenten

{'bwl_ges': 3132,
 'bwl_w': 1397,
 'jura_ges': 1442,
 'jura_w': 787,
 'philo_ges': 2105,
 'philo_w': 1581,
 'sozi_ges': 1525,
 'sozi_w': 1047,
 'vwl_ges': 848,
 'vwl_w': 227,
 'wiinf_ges': 1129,
 'wiinf_w': 343}

In [12]:
studenten = int(dict_anzahl_studenten['bwl_ges']) + int(dict_anzahl_studenten['philo_ges']) + int(dict_anzahl_studenten['jura_ges']) + int(dict_anzahl_studenten['vwl_ges']) + int(dict_anzahl_studenten['wiinf_ges']) + int(dict_anzahl_studenten['sozi_ges'])
studenten

10181

# Abfrage Prüfungen

In [13]:
URL = 'https://www.uni-mannheim.de/pruefungstermine/pruefungen-ersttermine-bachelor/' 
website = requests.get(URL)
soup = BeautifulSoup(website.text, 'html.parser')

list=soup.find_all('div',{'class': 'tx-uma-portal-exam-category'})

prüfungen = 0


for i in list:
  print(i.find('h2'))
  for t in (i.find_all('td')):
    if "2022" in t.text:
      print(t.text)
      # Aufaddieren der Prüfungen am aktuellen Tag
      dates_test=t.text
      dates_prüfungen = datetime.strptime(dates_test, '%d.%m.%Y')
      if today == dates_prüfungen:
        prüfungen += 1

<h2>Bachelor Anglistik/<wbr/>Amerikanistik / CELLS</h2>
04.06.2022
07.06.2022
07.06.2022
08.06.2022
08.06.2022
09.06.2022
09.06.2022
10.06.2022
10.06.2022
10.06.2022
11.06.2022
11.06.2022
11.06.2022
13.06.2022
13.06.2022
13.06.2022
14.06.2022
14.06.2022
14.06.2022
15.06.2022
15.06.2022
15.06.2022
17.06.2022
17.06.2022
17.06.2022
18.06.2022
<h2>Bachelor Beifach Öffentliches Recht</h2>
09.06.2022
13.06.2022
<h2>Bachelor Beifach Psychologie</h2>
04.06.2022
04.06.2022
07.06.2022
08.06.2022
09.06.2022
<h2>Bachelor Betriebs­wirtschafts­lehre 2. Semester</h2>
04.06.2022
08.06.2022
14.06.2022
17.06.2022
<h2>Bachelor Betriebs­wirtschafts­lehre 4. Semester</h2>
06.04.2022
07.04.2022
08.04.2022
08.04.2022
27.04.2022
06.05.2022
20.05.2022
20.05.2022
07.06.2022
09.06.2022
10.06.2022
10.06.2022
10.06.2022
11.06.2022
11.06.2022
13.06.2022
13.06.2022
15.06.2022
18.06.2022
18.06.2022
<h2>Bachelor Betriebs­wirtschafts­lehre 6. Semester</h2>
28.03.2022
30.03.2022
01.04.2022
08.06.2022
13.06.2022
<h2>Bach

In [14]:
prüfungen

0

# Geldfaktor

In [15]:
#In der Regel bekommt man Ende des Montas das gehalt
gehalt = str(25)+'-'+str(month)+'-'+str(year)
gehalt = datetime.strptime(gehalt,'%d-%m-%Y').date()
#In der Mitte des Monats ist am wenigsten Geld vorhanden 
gehalt_weg = str(15)+'-'+str(month)+'-'+str(year)
gehalt_weg = datetime.strptime(gehalt_weg,'%d-%m-%Y').date()

if today > gehalt or today < gehalt_weg: 
  geldfaktor = 5
else:
  geldfaktor = -5
geldfaktor

5

## Barometererrechnung 

In [28]:
base = 100

# Geldfaktor
base += geldfaktor
print(base)
# Anzahl Prüfungen --> Gewichtung Prüfungen mal Studenten mit einer Skalierung von 10000, da die Studenten im 10000 Bereich sind 
prüfungen = prüfungen / studenten *10000
base += prüfungen
print(base)

# Events --> Faktor 10 um die Events stärker zu gewichten
events_value = events * 10 
base += events_value
print(base)

# Wetterfaktor --> Faktor 10 um das Wetter stärker zu gewichten
weather_value = b * 10
base -= weather_value
print(base)


105
105.0
115.0
115.0


In [17]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2022-08-12 02:40:09--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  19.2MB/s    in 0.7s    

2022-08-12 02:40:10 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [18]:
!pip install dash
!pip install dash_html_components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc

import plotly.graph_objects as go
trace1 = go.Indicator(mode="gauge+number",    
                      value=base,    domain={'x': [0.0, 0.4], 'y': [0.0, 1]},    
                      title={'text': 'Partybarometer'},
                      gauge = {'axis': {'range': [None, 1]}})
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = base,
    mode = "gauge+number+delta",
    title = {'text': "Speed"},
    delta = {'reference': 100},
    gauge = {'axis': {'range': [None, base+100]},
             'steps' : [
                 {'range': [0, 250], 'color': "lightgray"},
                 {'range': [250, 400], 'color': "gray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    # All elements from the top of the page
    html.Div([
        html.H1(children='Barometer'),

        html.Div(children='''
            Das Barometer hat folgendes ermittelt.
        '''),

        dcc.Graph(
            id='graph1',
            figure=fig
        ),
    ]),
])
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>